In [1]:
import sqlite3

In [2]:
before_db_conn = sqlite3.connect('serach_index.db')


In [3]:
before_db_cur = before_db_conn.cursor()

In [4]:
sql = '''
SELECT * FROM headline
INNER JOIN headline_paths ON headline.id = headline_paths.id
INNER JOIN headline_size ON headline.id = headline_size.id
INNER JOIN ocr_text ON headline.id = ocr_text.id
WHERE headline.id < 10;
'''

In [5]:
result = before_db_cur.execute(sql)

In [6]:
import pprint

for r in result:
    pprint.pprint(r)
    print('=' * 20)

(0,
 'こども大分新聞',
 '1932年',
 '10月',
 '2-y-2_01',
 'んぐみ\n| day\n三人組の隨一 ジ一堅い信念を貫いた。\nBEATION FLOR ANATUREHOU\n||||',
 0,
 'こども大分新聞/1932年/10月/2-y-2_01.jpg',
 0,
 258,
 478,
 0,
 'んぐみ\n| day\n三人組の隨一 ジ一堅い信念を貫いた。\nBEATION FLOR ANATUREHOU\n||||')
(1,
 'こども大分新聞',
 '1932年',
 '10月',
 '2-y-2_02',
 '四戒を守って 健やか&伸びる 專科に運動に大いに養\n日田男子校の巻',
 1,
 'こども大分新聞/1932年/10月/2-y-2_02.jpg',
 1,
 172,
 314,
 1,
 '四戒を守って 健やか&伸びる 專科に運動に大いに養\n日田男子校の巻')
(2,
 'こども大分新聞',
 '1932年',
 '10月',
 '2-y-2_03',
 '少年 少 女 講 座 メートル法の起源',
 2,
 'こども大分新聞/1932年/10月/2-y-2_03.jpg',
 2,
 98,
 280,
 2,
 '少年 少 女 講 座 メートル法の起源')
(3,
 'こども大分新聞',
 '1932年',
 '10月',
 '2-y-9_01',
 '10WH\n||||||||RUNWHIEF\n電器具と 附屬品のいろく\n文明國の小國民は\n是非知って置きませう',
 3,
 'こども大分新聞/1932年/10月/2-y-9_01.jpg',
 3,
 280,
 452,
 3,
 '10WH\n||||||||RUNWHIEF\n電器具と 附屬品のいろく\n文明國の小國民は\n是非知って置きませう')
(4,
 'こども大分新聞',
 '1932年',
 '10月',
 '2-y-9_04',
 'hateway.\nいろいろ\n電燈線の',
 4,
 'こども大分新聞/1932年/10月/2-y-9_04.jpg',
 4,
 94,
 150,
 4,
 'hateway.\nいろいろ\n電燈線の')
(5,
 'こども大分新聞',
 '1932年',
 '10月',
 '2-

In [7]:
sql = '''
SELECT headline.id, company, year, month, page, headline_text, width, height, path FROM headline
INNER JOIN headline_paths ON headline.id = headline_paths.id
INNER JOIN headline_size ON headline.id = headline_size.id
INNER JOIN ocr_text ON headline.id = ocr_text.id
WHERE headline.id < 10;
'''


In [8]:
result = before_db_cur.execute(sql)


In [9]:
for id, company, year, month, page, text, width, height, path in result:
    print(f'{id}, {company}-{year}-{month}-{page}')
    print(f'path: {path}')
    print(f'width: {width}, height: {height}')
    print(text)

    print('*' * 50)


0, こども大分新聞-1932年-10月-2-y-2_01
path: こども大分新聞/1932年/10月/2-y-2_01.jpg
width: 258, height: 478
んぐみ
| day
三人組の隨一 ジ一堅い信念を貫いた。
BEATION FLOR ANATUREHOU
||||
**************************************************
1, こども大分新聞-1932年-10月-2-y-2_02
path: こども大分新聞/1932年/10月/2-y-2_02.jpg
width: 172, height: 314
四戒を守って 健やか&伸びる 專科に運動に大いに養
日田男子校の巻
**************************************************
2, こども大分新聞-1932年-10月-2-y-2_03
path: こども大分新聞/1932年/10月/2-y-2_03.jpg
width: 98, height: 280
少年 少 女 講 座 メートル法の起源
**************************************************
3, こども大分新聞-1932年-10月-2-y-9_01
path: こども大分新聞/1932年/10月/2-y-9_01.jpg
width: 280, height: 452
10WH
||||||||RUNWHIEF
電器具と 附屬品のいろく
文明國の小國民は
是非知って置きませう
**************************************************
4, こども大分新聞-1932年-10月-2-y-9_04
path: こども大分新聞/1932年/10月/2-y-9_04.jpg
width: 94, height: 150
hateway.
いろいろ
電燈線の
**************************************************
5, こども大分新聞-1932年-10月-2-y-4_01
path: こども大分新聞/1932年/10月/2-y-4_01.jpg
width: 178, height: 318
日本は殊に 霞が招はい それは

# モデル化

In [10]:
from sqlalchemy import Table, Column, Integer, String, ForeignKey, DateTime
from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql.functions import current_timestamp
from sqlalchemy.sql.sqltypes import NullType


In [11]:
# DB接続
engine = create_engine('sqlite:///search_index2.db')

Base = declarative_base()


In [12]:
from datetime import datetime

In [13]:
class Headline(Base):
    """Headlineモデル
    見出しを管理するテーブル

    Args:
        Base (sqlalchemy.orm.decl_api.DeclarativeMeta): ORMのベース
    """
    __tablename__ = 'headlines'

    '''見出しID(主キー)'''
    id = Column(Integer)
    id.primary_key = True
    '''新聞社名'''
    company = Column(String)
    '''発刊年'''
    year = Column(String)
    '''発刊月'''
    month = Column(String)
    '''発刊日+ページ'''
    page = Column(String)
    '''見出し画像パス'''
    path = Column(String)
    '''見出し画像の幅'''
    width = Column(Integer)
    '''見出し画像の高さ'''
    height = Column(Integer)
    '''見出し画像から抽出したテキスト'''
    text = Column(String)
    # '''作成日'''
    # created_at = Column(DateTime)
    # created_at.server_default = current_timestamp()
    # created_at.nullable = False
    # '''更新日'''
    # updated_at = Column(DateTime)
    # updated_at.server_default = current_timestamp()
    # updated_at.onupdate = current_timestamp()
    # updated_at.nullable = False

In [14]:
Base.metadata.create_all(engine)

In [15]:
# セッションの生成
Session = sessionmaker(bind=engine)
session = Session()


# データベースに格納

In [16]:
sql = '''
SELECT headline.id, company, year, month, page, headline_text, width, height, path FROM headline
INNER JOIN headline_paths ON headline.id = headline_paths.id
INNER JOIN headline_size ON headline.id = headline_size.id
INNER JOIN ocr_text ON headline.id = ocr_text.id;
'''


In [17]:
result = before_db_cur.execute(sql)


In [18]:
headlines = []
for id, company, year, month, page, text, width, height, path in result:
    headlines.append(Headline(
        id=id,
        company=company,
        year=year,
        month=month,
        page=page,
        text=text,
        width=width,
        height=height,
        path=path
        ))


In [19]:
for i, headline in enumerate(headlines):
    if i < 5:
        pprint.pprint(vars(headline))

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x00000216DEA5DBA0>,
 'company': 'こども大分新聞',
 'height': 478,
 'id': 0,
 'month': '10月',
 'page': '2-y-2_01',
 'path': 'こども大分新聞/1932年/10月/2-y-2_01.jpg',
 'text': 'んぐみ\n| day\n三人組の隨一 ジ一堅い信念を貫いた。\nBEATION FLOR ANATUREHOU\n||||',
 'width': 258,
 'year': '1932年'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x00000216DEA5E860>,
 'company': 'こども大分新聞',
 'height': 314,
 'id': 1,
 'month': '10月',
 'page': '2-y-2_02',
 'path': 'こども大分新聞/1932年/10月/2-y-2_02.jpg',
 'text': '四戒を守って 健やか&伸びる 專科に運動に大いに養\n日田男子校の巻',
 'width': 172,
 'year': '1932年'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x00000216DEA5DED0>,
 'company': 'こども大分新聞',
 'height': 280,
 'id': 2,
 'month': '10月',
 'page': '2-y-2_03',
 'path': 'こども大分新聞/1932年/10月/2-y-2_03.jpg',
 'text': '少年 少 女 講 座 メートル法の起源',
 'width': 98,
 'year': '1932年'}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x00000216DEA5D1E0>,

In [20]:
max = len(headlines)

In [21]:
max

1253950

In [22]:
session.add_all(headlines)

In [23]:
session.commit()


### 失敗テイク
created_atとか設定するとなんかcommitできなかった

In [33]:
session.rollback() # エラーでた...

In [34]:
for headline in headlines:
    session.add(headline)


In [36]:
session.commit()

TypeError: Boolean value of this clause is not defined

In [37]:
headlines[0].created_at


In [38]:
current_timestamp()

<sqlalchemy.sql.functions.current_timestamp at 0x1fb792f6050; current_timestamp>

# 呼び出しテスト

In [24]:
headlines = session.query(Headline).limit(100).all()

In [27]:
for headline in headlines:
    print(f'{headline.path}')
    print(f'width: {headline.width}, height: {headline.height}')
    print(headline.text)
    print('==' * 20)

こども大分新聞/1932年/10月/2-y-2_01.jpg
width: 258, height: 478
んぐみ
| day
三人組の隨一 ジ一堅い信念を貫いた。
BEATION FLOR ANATUREHOU
||||
こども大分新聞/1932年/10月/2-y-2_02.jpg
width: 172, height: 314
四戒を守って 健やか&伸びる 專科に運動に大いに養
日田男子校の巻
こども大分新聞/1932年/10月/2-y-2_03.jpg
width: 98, height: 280
少年 少 女 講 座 メートル法の起源
こども大分新聞/1932年/10月/2-y-9_01.jpg
width: 280, height: 452
10WH
||||||||RUNWHIEF
電器具と 附屬品のいろく
文明國の小國民は
是非知って置きませう
こども大分新聞/1932年/10月/2-y-9_04.jpg
width: 94, height: 150
hateway.
いろいろ
電燈線の
こども大分新聞/1932年/10月/2-y-4_01.jpg
width: 178, height: 318
日本は殊に 霞が招はい それは気温を 氣壁の配置から」
こども大分新聞/1932年/10月/2-a-8_01.jpg
width: 50, height: 210
の 發 明
こども大分新聞/1932年/10月/2-a-8_03.jpg
width: 54, height: 242
でんわ
くった
電話機の組立
こども大分新聞/1932年/10月/2-y-6_02.jpg
width: 158, height: 596
たましひ
1日本魂の權化
―肉弾三勇士の少年時代―
ゆう
こども大分新聞/1932年/10月/2-y-9_02.jpg
width: 142, height: 304
照渡自在な。 スウィッチー
その變り種のいろく
こども大分新聞/1932年/10月/1-y-2_01.jpg
width: 256, height: 458
おとな
・大人も及ば。
非凡な度胸と動
こども大分新聞/1932年/10月/2-y-6_01.jpg
width: 140, height: 436
とな
たけじく
負け嫌ひの伊之助君 温和しい武二君
風變リの子供―釣の好

In [30]:
# Likeによる部分一致呼び出し
headlines = session.query(Headline).filter(Headline.text.like('%大分%')).limit(10).all()

In [31]:
for headline in headlines:
    print(f'{headline.path}')
    print(f'width: {headline.width}, height: {headline.height}')
    print(headline.text)
    print('==' * 20)


こども大分新聞/1932年/10月/2-y-4_03.jpg
width: 70, height: 138
兄さん 大分郡松岡校
こども大分新聞/1932年/10月/1-y-2_02.jpg
width: 224, height: 310
ゅれらの學校2 校風確立の礎 一徳園と一心屬
織島材の堅識な意氣 大分郡植田校の巻
こども大分新聞/1932年/11月/11-a-4_02.jpg
width: 0, height: 0
推獎二篇 エンソク 大分師範附脇校 「靜一財前司之子
こども大分新聞/1932年/11月/11-a-8_03.jpg
width: 0, height: 0
大分縣だけで 昨年は五十九萬 一千夏が煙になつた
こども大分新聞/1932年/11月/4-a-2_01.jpg
width: 0, height: 0
大分縣 / 光榮の御學友
陽平民的女殿樣
不然一
大分の児童へ御土産」 蕃府內藩主大給近孝子爵
大津和夫 ov” “ u ,
こども大分新聞/1932年/1月/10-a-2_01.jpg
width: 280, height: 112
小名
時代,
大分縣
こども大分新聞/1932年/1月/17-a-8_03.jpg
width: 100, height: 408
電燈の知識
九水大分営業所安部二
こども大分新聞/1932年/1月/17-a-2_02.jpg
width: 126, height: 260
少年少女 講座、
はなし
そうせんきょ
總選舉の話
大分新聞記者衛藤
こども大分新聞/1932年/1月/10-a-8_01.jpg
width: 226, height: 470
海岸線百八十里! 大分縣の水産業
覚ましい遠洋漁業や養殖の話
なし
ぎょぎょう %子供の科學|魚業
CON
こども大分新聞/1932年/1月/24-a-2_02.jpg
width: 146, height: 304
少年 少女講 座 =
そうせんきょ
總選舉の話
大分新聞衛藤原
